In [ ]:
import json

## Load calendars

In [ ]:
with open("/Users/valentin/Projets/prono_foot/data/crawl_soccerway_FR/calendar/%s.json" % "calendar_1990_1996", "r") as f:
    calendar = json.load(f)
    f.close()

### Find unique competitions

In [ ]:
competitions = list()
for day in calendar :
    for competition in calendar[day]:
        c = competition.split(" - ")
        compet = {
            "region": c[0],
            "name": " - ".join(c[1:])
        }
        if compet not in competitions:
            competitions.append(compet)


In [ ]:

print len(competitions)

regions = set()
for competition in competitions:
    regions.add(competition["region"])
print len(regions)
print regions

In [ ]:
from betlib.models.competition import add_competition
for competition in competitions:
    add_competition(competition)

## Load games

In [ ]:
with open("/Users/valentin/Projets/prono_foot/data/crawl_soccerway_FR/games/%s.json" % "games_2005", "r") as f:
    games = json.load(f)
    f.close()

In [ ]:
regions = set()
for i, game in enumerate(games):
    if i == 2110:
        print game
    regions.add(game["country"])
print len(games)

In [ ]:
print len(regions)

In [ ]:
print regions

In [ ]:
from betlib.models.region import find_region, add_region_alias, find_region_by_name

for region in regions:
    if find_region_by_name(region) == None:
        print region

In [ ]:
add_region_alias("South America", u'Am\xe9rique du Sud')

In [ ]:
from betlib.models.team import add_team

teams = list()
for game in games:
    if game["team_H"] not in teams:
        teams.append(game["team_H"])
        add_team(game["team_H"])
    if game["team_A"] not in teams:
        teams.append(game["team_A"])
        add_team(game["team_A"])

In [ ]:
from betlib.models.people import add_people

none_nb = 0
peoples = list()
for game in games:
        for team in game['teams']:
            for status in game['teams'][team]:
                if game['teams'][team][status] is None:
                        none_nb += 1
                else:
                    for people in game['teams'][team][status]:
                        if people not in peoples:
                            peoples.append(people)
                            if "name" in people and "id" in people:
                                add_people({"name": people["name"], "link": "http://fr.soccerway" + people["id"]})

print len(peoples)

In [ ]:
from betlib.models.stadium import add_stadium

for game in games:
    if "stadium" in game and game["stadium"] is not None:
        print game["stadium"]
        add_stadium({"name": game["stadium"]})

## Add game in db

In [ ]:
from betlib.models.game import add_game

for game in games:
    if "half-time" in game["scores"] and "final" not in game["scores"]:
        game["scores"]["final"] = game["scores"]["half-time"]
        game["scores"]["half-time"] = " - "
    add_game({
            "link": ["http://fr.soccerway.com" + game["game_link"]],
            "team_H": game["team_H"]["name"],
            "team_A": game["team_A"]["name"],
            "score": {
                "half_time": {
                    "home": game["scores"]["half-time"].split(" - ")[0],
                    "away": game["scores"]["half-time"].split(" - ")[1],
                },
                "final": {
                    "home": game["scores"].get("final", "").split(" - ")[0],
                    "away": game["scores"].get("final", " - ").split(" - ")[1],
                }
            },
            "stadium": game["stadium"],
            "start-time": game["start_time"],
            "date": game["date"],
            "region": game["country"],
            "division": game["division"],
        })

In [ ]:
print len(games)
